# CommonLit - Predicting Readability

In this notebook I create an algorithm that predicts the readability of texts from the CommonLit dataset.

I proceed as follows:
1. Extract 3 Test sets from the Training dataset
2. Make 10-fold validation splits using the remaing Training data
3. Apply a TF-IDF word embedding to all texts
4. Apply a neural network on the word embedded texts
5. Make 10 predictions on each Test dataset
6. Compute the Mean and Median of all 10 predictions, for each Test dataset

On the 3 Test Datasets (explained in step 1) I get $R^2s$ ranging between 0.85 and 0.92 and $RMSEs$ between 0.5 and 0.8.

I get a $RMSE$ submission score of 0.750.

In [1]:
# Import Data

import pandas as pd

Original_Train_Data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
Test_Data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
Sample_Submission = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

I implement a Train-Test split to extract 3 Test Data Sets from the Train Data, with 7 observations in each test set.

In [2]:
from sklearn.model_selection import train_test_split

Train_Data, NewTest_Data = train_test_split(Original_Train_Data, test_size=0.009, random_state=59)

Test_Data1, Test_Data2 = train_test_split(NewTest_Data, test_size=0.33, random_state=39)

Test_Data1, Test_Data3 = train_test_split(Test_Data1, test_size=0.5, random_state=17)

Test_Data1 = Test_Data1.reset_index()
Test_Data2 = Test_Data2.reset_index()
Test_Data3 = Test_Data3.reset_index()

I define a function that:
* does the pre-treatment of all texts
* applies a TF-IDF to the previously the pre-treated texts
* defines a neural network
* returns a prediction for each test dataset

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from keras.layers import Dropout
from transformers import RobertaTokenizer
import keras

def TFIDF_NeuralNetwork_Prediction(Data_Train, Data_Validation, Data_Test, Data_Test1, Data_Test2, Data_Test3):
    
    # Store each excertp into lists: Train_Corpus, Validation_Corpus, Test_Corpus
    
    Data_Train = Data_Train.reset_index()
    Data_Validation = Data_Validation.reset_index()
    
    Data_Test = Data_Test.reset_index()
    Data_Test1 = Data_Test1.reset_index()
    Data_Test2 = Data_Test2.reset_index()
    Data_Test3 = Data_Test3.reset_index()
    
    Train_Corpus = []
    for j in range(len(Data_Train)):
        Train_Corpus.append(str(Data_Train['excerpt'].iloc[j]))
        
    Validation_Corpus = []
    for j in range(len(Data_Validation)):
        Validation_Corpus.append(str(Data_Validation['excerpt'].iloc[j]))
        
    Test_Corpus = []
    for j in range(len(Data_Test)):
        Test_Corpus.append(str(Data_Test['excerpt'].iloc[j]))
        
    Test_Corpus1 = []
    for j in range(len(Data_Test1)):
        Test_Corpus1.append(str(Data_Test1['excerpt'].iloc[j]))
        
    Test_Corpus2 = []
    for j in range(len(Data_Test2)):
        Test_Corpus2.append(str(Data_Test2['excerpt'].iloc[j]))
        
    Test_Corpus3 = []
    for j in range(len(Data_Test3)):
        Test_Corpus3.append(str(Data_Test3['excerpt'].iloc[j]))
        
    # Fit TF-IDF on the Training_Corpus, then transfrom Validation_Corpus, Test
    
    TFIDF = TfidfVectorizer(analyzer='word',stop_words= 'english')
    
    TFIDF.fit(Train_Corpus)
    
    TFIDF_Train = TFIDF.transform(Train_Corpus)
    TFIDF_Validation = TFIDF.transform(Validation_Corpus)
    
    TFIDF_Test = TFIDF.transform(Test_Corpus)
    TFIDF_Test1 = TFIDF.transform(Test_Corpus1)
    TFIDF_Test2 = TFIDF.transform(Test_Corpus2)
    TFIDF_Test3 = TFIDF.transform(Test_Corpus3)
    
    
    # Store TF-IDFs in DataFrames

    TFIDF_DataTrain = pd.DataFrame(data = TFIDF_Train.toarray(),columns = TFIDF.get_feature_names())
    TFIDF_DataValidation = pd.DataFrame(data = TFIDF_Validation.toarray(),columns = TFIDF.get_feature_names())
    
    TFIDF_DataTest = pd.DataFrame(data = TFIDF_Test.toarray(),columns = TFIDF.get_feature_names())
    TFIDF_DataTest1 = pd.DataFrame(data = TFIDF_Test1.toarray(),columns = TFIDF.get_feature_names())
    TFIDF_DataTest2 = pd.DataFrame(data = TFIDF_Test2.toarray(),columns = TFIDF.get_feature_names())
    TFIDF_DataTest3 = pd.DataFrame(data = TFIDF_Test3.toarray(),columns = TFIDF.get_feature_names())
    
    
    # Build Model
    
    model = keras.models.Sequential([
        keras.layers.Dense(2000, activation="relu", input_shape=TFIDF_Train.shape),       
        keras.layers.Dropout(0.1),                            
        keras.layers.Dense(1000, activation="relu"), 
        keras.layers.Dropout(0.1),                               
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(250, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(125, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(60, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(30, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(15, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(1)
    ])
    
    # Fit Model on Train data, validate on validation data, and predict on Test_Data
    
    model.compile(loss="mse", optimizer="sgd")
    history = model.fit(TFIDF_DataTrain , Data_Train['target'], epochs=150,
                        validation_data=(TFIDF_DataValidation , Data_Validation['target']))
    
    Predictions_Test = model.predict(TFIDF_DataTest)
    Predictions_Test1 = model.predict(TFIDF_DataTest1)
    Predictions_Test2 = model.predict(TFIDF_DataTest2)
    Predictions_Test3 = model.predict(TFIDF_DataTest3)
    
    return Predictions_Test, Predictions_Test1, Predictions_Test2, Predictions_Test3

I implement a 10-fold Cross-Validation split, and then run the previously defined function that applies a TF-IDF, and then a neural network to make 10 predictions for every text on the 3 *Test sets*.

In [4]:
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

kf = KFold(n_splits=10,random_state=123, shuffle=True)

Split = -1
for train_index, validation_index in kf.split(Train_Data):
    Split = Split + 1
    X_Train, X_Validation = Train_Data.iloc[train_index], Train_Data.iloc[validation_index]
    Test_Data['Prediction_'+str(Split)], Test_Data1['Prediction_'+str(Split)], Test_Data2['Prediction_'+str(Split)], Test_Data3['Prediction_'+str(Split)] = TFIDF_NeuralNetwork_Prediction(X_Train, X_Validation, Test_Data, Test_Data1, Test_Data2, Test_Data3)

Epoch 1/150
79/79 [==============================] - 3s 17ms/step - loss: 1.3735 - val_loss: 1.1436
Epoch 2/150
79/79 [==============================] - 1s 9ms/step - loss: 1.1110 - val_loss: 1.1427
Epoch 3/150
79/79 [==============================] - 1s 10ms/step - loss: 1.1063 - val_loss: 1.1435
Epoch 4/150
79/79 [==============================] - 1s 9ms/step - loss: 1.0746 - val_loss: 1.1418
Epoch 5/150
79/79 [==============================] - 1s 9ms/step - loss: 1.0663 - val_loss: 1.1396
Epoch 6/150
79/79 [==============================] - 1s 9ms/step - loss: 1.0709 - val_loss: 1.1383
Epoch 7/150
79/79 [==============================] - 1s 9ms/step - loss: 1.0282 - val_loss: 1.1383
Epoch 8/150
79/79 [==============================] - 1s 9ms/step - loss: 1.0603 - val_loss: 1.1285
Epoch 9/150
79/79 [==============================] - 1s 9ms/step - loss: 1.0012 - val_loss: 1.1114
Epoch 10/150
79/79 [==============================] - 1s 9ms/step - loss: 1.0139 - val_loss: 1.0819
Epoch 1

I define functions to calculate evaluation metrics: $R^2$ and $RMSE$.

In [5]:
# Define functions which compute evaluation scores (R2 and RMSE)

import numpy as np

def R2(RealY,PredictedY):
    Top = (RealY - PredictedY)**2
    MeanY = np.ones(len(RealY))
    Bottom = (RealY - MeanY)**2
    R2 = 1 - Top.sum() / Bottom.sum()
    return R2

def RMSE(RealY,PredictedY):
    RMSE = (((RealY - PredictedY)**2).mean())**(1/2)
    return RMSE

Having produced 10 predictions for each text, I create two functions to aggregate the 10 predictions:
* Median of the 10 predictions
* Mean of the 10 predictions

In [6]:
import statistics

def MedianPrediction(DataFrame, nb_predictions):
    MedianPredictions = []
    for i in range(len(DataFrame)):
        Predictions = []
        for j in range(nb_predictions):
            VariableName = 'Prediction_' + str(j)
            Predictions.append(DataFrame[VariableName].iloc[i])
        MedianPredictions.append(statistics.median(Predictions))
    return MedianPredictions


def MeanPrediction(DataFrame, nb_predictions):
    DataFrame['MeanPrediction'] = 0
    for j in range(nb_predictions):
        VariableName = 'Prediction_' + str(j)
        DataFrame['MeanPrediction'] = DataFrame['MeanPrediction']  +  DataFrame[VariableName]
    DataFrame['MeanPrediction'] = DataFrame['MeanPrediction'] / nb_predictions
    MeanPrediction =   DataFrame['MeanPrediction'].to_list()
    return MeanPrediction

I apply the 2 previously defined functions to orginal test data set (which should be used for the submission), and to the 3 test data sets I have created in the second cell of this notebook.

In [7]:
# Compute Mean and Median of all 10 predictions - Original test dataset (for submission)

Test_Data['MedianPredictions'] = MedianPrediction(Test_Data, 10)

Test_Data['MeanPredictions'] = MeanPrediction(Test_Data, 10)

Test_Data['FinalPredictions'] = 0.5*Test_Data['MedianPredictions'] + 0.5*Test_Data['MeanPredictions']


# Compute Mean and Median of all 10 predictions - Test1, Test2 and Test3 datasets

Test_Data1['MedianPredictions'] = MedianPrediction(Test_Data1, 10)
Test_Data2['MedianPredictions'] = MedianPrediction(Test_Data2, 10)
Test_Data3['MedianPredictions'] = MedianPrediction(Test_Data3, 10)

Test_Data1['MeanPredictions'] = MeanPrediction(Test_Data1, 10)
Test_Data2['MeanPredictions'] = MeanPrediction(Test_Data2, 10)
Test_Data3['MeanPredictions'] = MeanPrediction(Test_Data3, 10)

Test_Data1['FinalPredictions'] = 0.5*Test_Data1['MedianPredictions'] + 0.5*Test_Data1['MeanPredictions']
Test_Data2['FinalPredictions'] = 0.5*Test_Data2['MedianPredictions'] + 0.5*Test_Data2['MeanPredictions']
Test_Data3['FinalPredictions'] = 0.5*Test_Data3['MedianPredictions'] + 0.5*Test_Data3['MeanPredictions']

Print evaluation metrics.

In [8]:
print('Median Predictions - R2 :')
print(" ")
print("Test 1 : ", R2(Test_Data1['target'],Test_Data1['MedianPredictions']))
print("Test 2 : ", R2(Test_Data2['target'],Test_Data2['MedianPredictions']))
print("Test 3 : ", R2(Test_Data3['target'],Test_Data3['MedianPredictions']))
print(" ")
print('Mean Predictions - R2 :')
print(" ")
print("Test 1 : ", R2(Test_Data1['target'],Test_Data1['MeanPredictions']))
print("Test 2 : ", R2(Test_Data2['target'],Test_Data2['MeanPredictions']))
print("Test 3 : ", R2(Test_Data3['target'],Test_Data3['MeanPredictions']))

print(" ")
print(" ")
print('Median Predictions - RMSE :')
print(" ")
print("Test 1 : ", RMSE(Test_Data1['target'],Test_Data1['MedianPredictions']))
print("Test 2 : ", RMSE(Test_Data2['target'],Test_Data2['MedianPredictions']))
print("Test 3 : ", RMSE(Test_Data3['target'],Test_Data3['MedianPredictions']))
print(" ")
print('Mean Predictions - RMSE :')
print(" ")
print("Test 1 : ", RMSE(Test_Data1['target'],Test_Data1['MeanPredictions']))
print("Test 2 : ", RMSE(Test_Data2['target'],Test_Data2['MeanPredictions']))
print("Test 3 : ", RMSE(Test_Data3['target'],Test_Data3['MeanPredictions']))

Median Predictions - R2 :
 
Test 1 :  0.9327955386495118
Test 2 :  0.859953513360809
Test 3 :  0.9078478262262196
 
Mean Predictions - R2 :
 
Test 1 :  0.9275339551979265
Test 2 :  0.8599021956979873
Test 3 :  0.909692035259746
 
 
Median Predictions - RMSE :
 
Test 1 :  0.5057717992967242
Test 2 :  0.8763421317193333
Test 3 :  0.6940016219379174
 
Mean Predictions - RMSE :
 
Test 1 :  0.5251977262368736
Test 2 :  0.8765026773784016
Test 3 :  0.687022120732726


Create the submission sample.

In [9]:
New_Sample_Submission = Sample_Submission.copy()
New_Sample_Submission['target'] = Test_Data['FinalPredictions']

New_Sample_Submission.to_csv('submission.csv',index=False)